In [7]:
from fastembed import TextEmbedding

model_name = 'BAAI/bge-small-en-v1.5'
emb_model = TextEmbedding(model_name=model_name)

In [8]:
def embed_fastembed_(input_text: str):
    embedding = list(emb_model.embed(input_text))[0].tolist()
    return embedding

In [9]:
from lancedb.pydantic import LanceModel, Vector
import lancedb


class DataModel(LanceModel):
    vector: Vector(dim=384)  # type: ignore
    text: str
    source: str
    page: str

db = lancedb.connect("./volumes/lancedb")
table = db.create_table("rag", schema=DataModel, mode="overwrite")

[2024-09-25T08:14:48Z WARN  lance::dataset] No existing dataset at /home/ubuntu/data/zoomdep/volumes/lancedb/rag.lance, it will be created


In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader('./data')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
texts = text_splitter.split_documents(docs)

In [11]:
len(texts)

924

In [12]:
from tqdm.auto import tqdm

In [13]:
docs = [{ "source": text.metadata['source'].split('/')[-1], "page": text.metadata['page'], "text": text.page_content, "vector": embed_fastembed_(text.page_content)  }  for text in tqdm(texts) ]

  0%|          | 0/924 [00:00<?, ?it/s]

In [14]:
table.add(data=docs)

In [15]:
table.count_rows()

924

In [16]:
import lancedb

db = lancedb.connect("./volumes/lancedb")
table = db.open_table("rag")

In [17]:
table.create_fts_index("text")
